In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
data = np.load('/home/blu/workspace/graduate_project/data/didi_chengdu/Jan2Apr/origin.npy', allow_pickle=True)

In [3]:
road_id = np.unique(data[:,0])
time = np.unique(data[:,1])

In [4]:
print(road_id)
print(road_id.shape)
print(time)
print(time.shape)

[281863 281864 281865 ..., 283536 283537 283538]
(1676,)
['2018-01-01 00:00:00' '2018-01-01 00:10:00' '2018-01-01 00:20:00' ...,
 '2018-04-30 23:30:00' '2018-04-30 23:40:00' '2018-04-30 23:50:00']
(17280,)


In [5]:
mat = np.zeros((road_id.shape[0], time.shape[0], 2))

In [6]:
road_dict = {id: id-281863 for id in road_id}
time_dict = {time[i]: i for i in range(time.shape[0])}

In [7]:
time_dict_inv = {i: time[i] for i in range(time.shape[0])}

In [8]:
for msg in tqdm(data):
    mat[road_dict[msg[0]]][time_dict[msg[1]]] = [msg[2], msg[3]]

100%|██████████| 27814256/27814256 [00:59<00:00, 464121.22it/s]


In [9]:
empty_idx = np.where(mat==0)
print(empty_idx)

(array([   0,    0,    0, ..., 1675, 1675, 1675]), array([6783, 6783, 6801, ..., 7076, 7077, 7077]), array([0, 1, 0, ..., 1, 0, 1]))


In [10]:
day = 0
tti = 0
speed = 0
for n in tqdm(range(len(empty_idx[0]))):
    if n%2 == 0:
        for i in range(120):
            road_i = empty_idx[0][n]
            t = empty_idx[1][n] % 144
            if mat[road_i][t + i * 144][0] != 0:
                tti += mat[road_i][t + i * 144][0]
                speed += mat[road_i][t + i * 144][1]
                day += 1
        tti /= day
        speed /= day
        mat[empty_idx[0][n]][empty_idx[1][n]] = [tti, speed]

100%|██████████| 2294048/2294048 [06:13<00:00, 6141.74it/s]


In [11]:
np.save('/home/blu/workspace/graduate_project/data/didi_chengdu/Jan2Apr/completion.npy', mat)